In [1]:
# Initial imports
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

import random
import urllib.request
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [2]:
base_url = "https://sofifa.com/players?offset="
offset = 0
columns = ['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall', 'Potential', 'Club', 
           'Club Logo', 'Value', 'Wage', 'Special']
data = DataFrame(columns=columns)
for offset in range(8):
    url = base_url + str(offset*80)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    table_body = soup.find('tbody')
    counter = 0
    for row in table_body.findAll('tr'):
        td = row.findAll('td')
        picture = td[0].find('img').get('data-src')
        pid = td[0].find('img').get('id')
        nationality = td[1].find('a').get('title')
        flag_img = td[1].find('img').get('data-src')
        name = td[1].findAll('a')[1].text
        age = td[2].find('div').text.strip()
        overall = td[3].text.strip()
        potential = td[4].text.strip()
        club = td[5].find('a').text
        club_logo = td[5].find('img').get('data-src')
        value = td[7].text
        wage = td[8].text
        special = td[17].text
        player_data = DataFrame([[pid, name, age, picture, nationality, flag_img, overall, 
                                  potential, club, club_logo, value, wage, special]])
        player_data.columns = columns
        data = data.append(player_data, ignore_index=True)
        counter+=1
    offset+=1
    data.to_csv('player_data.csv', encoding='utf-8')

In [3]:
data

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special
0,20801,Cristiano Ronaldo,31,https://cdn.sofifa.org/48/17/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/17/teams/243.png,€87M,€340K,2225
1,167495,M. Neuer,30,https://cdn.sofifa.org/48/17/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/17/teams/21.png,€69.5M,€240K,1493
2,190871,Neymar,24,https://cdn.sofifa.org/48/17/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/17/teams/73.png,€106M,€260K,2098
3,188545,R. Lewandowski,27,https://cdn.sofifa.org/48/17/players/188545.png,Poland,https://cdn.sofifa.org/flags/37.png,91,91,FC Bayern Munich,https://cdn.sofifa.org/24/17/teams/21.png,€79.5M,€300K,2143
4,193080,De Gea,25,https://cdn.sofifa.org/48/17/players/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,90,92,Manchester United,https://cdn.sofifa.org/24/17/teams/11.png,€68.5M,€170K,1458
5,155862,Sergio Ramos,30,https://cdn.sofifa.org/48/17/players/155862.png,Spain,https://cdn.sofifa.org/flags/45.png,90,90,Real Madrid CF,https://cdn.sofifa.org/24/17/teams/243.png,€49M,€160K,2155
6,183277,E. Hazard,25,https://cdn.sofifa.org/48/17/players/183277.png,Belgium,https://cdn.sofifa.org/flags/7.png,90,91,Chelsea,https://cdn.sofifa.org/24/17/teams/5.png,€80M,€180K,2092
7,167664,G. Higuaín,28,https://cdn.sofifa.org/48/17/players/167664.png,Argentina,https://cdn.sofifa.org/flags/52.png,90,90,Juventus,https://cdn.sofifa.org/24/17/teams/45.png,€69M,€210K,1964
8,182521,T. Kroos,26,https://cdn.sofifa.org/48/17/players/182521.png,Germany,https://cdn.sofifa.org/flags/21.png,90,90,Real Madrid CF,https://cdn.sofifa.org/24/17/teams/243.png,€69.5M,€200K,2164
9,177003,L. Modrić,30,https://cdn.sofifa.org/48/17/players/177003.png,Croatia,https://cdn.sofifa.org/flags/10.png,89,89,Real Madrid CF,https://cdn.sofifa.org/24/17/teams/243.png,€54M,€160K,2231


In [98]:
player_data_url = 'https://sofifa.com/player/'
r = 0
for index, row in data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'col-3'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    master_data = DataFrame(columns=skill_names)
    break

In [100]:
player_data_url = 'https://sofifa.com/player/'
r = 0
for index, row in data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'col-3'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    attr_data = DataFrame(columns=skill_names)
    for key in skill_map.keys():
        attr_data.loc[r,key] = skill_map[key]
    r = r + 1
    master_data = pd.concat([master_data, attr_data])

In [101]:
master_data

,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,...,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys
0,91,63,90,63,93,93,84,81,92,93,...,96,83,92,23,92,92,31,80,85,88
1,58,29,52,35,48,70,15,14,30,13,...,85,55,25,11,61,44,10,83,70,11
2,93,56,96,82,95,80,75,81,96,89,...,88,81,78,33,90,79,24,53,80,83
3,79,80,78,80,89,87,62,77,85,91,...,91,83,88,19,83,79,42,84,78,87
4,57,38,60,43,42,64,17,21,18,13,...,88,50,31,13,58,40,21,64,68,13
5,77,84,80,60,84,76,66,73,61,60,...,85,78,79,90,78,84,89,81,63,66
6,93,54,92,90,92,86,80,82,93,83,...,85,86,79,22,87,77,27,65,86,79
7,77,50,75,69,85,86,68,74,84,91,...,88,75,88,18,81,75,22,85,70,88
8,57,60,70,69,89,85,85,85,79,76,...,86,90,87,69,52,78,82,74,88,82
9,77,62,93,94,92,84,78,79,86,71,...,88,92,73,73,71,83,80,58,90,74


In [103]:
master_data.to_csv('PlayerAttributeData.csv', encoding='utf-8')

In [105]:
full_data = pd.merge(data, master_data, left_index=True, right_index=True)

In [117]:
full_data.to_csv('FullDataset.csv', encoding='utf-8')